<a href="https://colab.research.google.com/github/eldinu1986m/mechera-detection/blob/main/MECHERA_DETECTION_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# CELDA 0: INSTALAR DEPENDENCIAS
!pip install streamlit pyngrok scikit-learn pandas numpy joblib -q
print("Dependencias instaladas: streamlit, pyngrok, scikit-learn, etc.")

Dependencias instaladas: streamlit, pyngrok, scikit-learn, etc.


In [11]:
# CELDA 1: MONTAR DRIVE
from google.colab import drive
import os

drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/mechera/models', exist_ok=True)
os.makedirs('/content/drive/MyDrive/mechera/data', exist_ok=True)

print("Drive montado. Carpetas listas: /mechera/models y /mechera/data")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive montado. Carpetas listas: /mechera/models y /mechera/data


In [12]:
# CELDA 2: DATOS SIMULADOS (RSSI + VENTAS)
import pandas as pd
import numpy as np

np.random.seed(42)
timestamps = pd.date_range('2023-10-01', periods=600, freq='100ms')

# RSSI
rssi = np.random.uniform(-90, -40, 600)
rssi_df = pd.DataFrame({'rssi': rssi}, index=timestamps)
rssi_df.to_csv('/content/drive/MyDrive/mechera/data/rssi.csv')

# Ventas
sales = np.random.uniform(0, 100, 600)
sales_df = pd.DataFrame({'sales': sales}, index=timestamps)
sales_df.to_csv('/content/drive/MyDrive/mechera/data/sales.csv')

print(f"Generados: {len(rssi_df)} RSSI | {len(sales_df)} ventas")

Generados: 600 RSSI | 600 ventas


In [13]:
# CELDA 3: FEATURES + MECHERAS
merged = rssi_df.join(sales_df)
merged['rssi_variance'] = merged['rssi'].rolling(3).var().fillna(0)
merged['dwell_proxy'] = merged['rssi'].rolling(3).mean().fillna(0)
merged['pass_count'] = (merged['rssi'].diff() != 0).cumsum()

# MECHERAS: lógica realista
mechera = pd.DataFrame({
    'rssi': np.random.uniform(-60, -40, 100),
    'rssi_variance': np.random.uniform(7, 12, 100),
    'dwell_proxy': np.random.uniform(-60, -45, 100),
    'pass_count': np.random.randint(6, 15, 100),
    'label_intention': ['potential_theft'] * 100
})

normal = pd.DataFrame({
    'rssi': np.random.uniform(-70, -50, 300),
    'rssi_variance': np.random.uniform(0, 4, 300),
    'dwell_proxy': np.random.uniform(-70, -55, 300),
    'pass_count': np.random.randint(1, 4, 300),
    'label_intention': ['browsing'] * 300
})

data = pd.concat([mechera, normal], ignore_index=True)
X = data[['rssi', 'rssi_variance', 'dwell_proxy', 'pass_count']]
y = data['label_intention']

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# Modelo intención
rf_intention = RandomForestClassifier(n_estimators=500, random_state=42)
rf_intention.fit(X, y)
joblib.dump(rf_intention, '/content/drive/MyDrive/mechera/models/rf_intention.pkl')

# Modelo ventas
X_sales = merged[['rssi', 'rssi_variance', 'dwell_proxy', 'pass_count']].dropna()
y_sales = merged['sales'].loc[X_sales.index]
rf_sales = RandomForestRegressor(n_estimators=100, random_state=42)
rf_sales.fit(X_sales, y_sales)
joblib.dump(rf_sales, '/content/drive/MyDrive/mechera/models/rf_sales.pkl')

print("Modelos entrenados: Mechera + Ventas")

Modelos entrenados: Mechera + Ventas


In [14]:
# CELDA 4: DASHBOARD
%%writefile /content/mechera_dashboard.py
import streamlit as st
import joblib
import numpy as np

# Cargar modelos
rf_sales = joblib.load('/content/drive/MyDrive/mechera/models/rf_sales.pkl')
rf_intention = joblib.load('/content/drive/MyDrive/mechera/models/rf_intention.pkl')

st.set_page_config(page_title="Mechera Detection", layout="centered")
st.title("MECHERA DETECTION SYSTEM")
st.markdown("**Ecofactoria Digital** - Detecta mecheras con WiFi del local")

col1, col2 = st.columns(2)
with col1:
    r = st.slider("RSSI (dBm)", -90., -40., -60.)
    v = st.slider("Varianza", 0., 15., 7.)
with col2:
    d = st.slider("Dwell", -90., -40., -55.)
    p = st.slider("Pasadas", 1, 15, 8)

if st.button("ANALIZAR CLIENTE", type="primary"):
    X = np.array([[r, v, d, p]])
    s = rf_sales.predict(X)[0]
    i = rf_intention.predict(X)[0]

    st.metric("Ventas estimadas", f"${s:.1f}")
    st.write(f"**Intención:** {i}")

    if i == 'potential_theft':
        st.error("MECHERA DETECTADA!")
        st.markdown("""
        <div style='text-align:center; padding:20px; background:#ffebee; border-radius:15px;'>
          <h1 style='color:red;'>¡ALERTA: POSIBLE MECHERA!</h1>
          <p>Movimiento sospechoso + muchas pasadas</p>
          <img src='https://media.giphy.com/media/3o7btPCcdNniyf0ArS/giphy.gif' width='250'>
        </div>
        """, unsafe_allow_html=True)
    else:
        st.success("Cliente normal - Sugerir producto")

Overwriting /content/mechera_dashboard.py


In [15]:
# CELDA 5: NGROK CON TU TOKEN
from pyngrok import ngrok
import time

# LIMPIAR
!pkill -f streamlit
!pkill -f ngrok
time.sleep(3)

# TU TOKEN (NO CAMBIAR)
!ngrok authtoken 34hWVmUmtYM1dy3U4lQYoq8XoFB_3r3v1ty8E87JHyTso22YN

print("ngrok configurado con tu token")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
ngrok configurado con tu token


In [ ]:
# CELDA 6: LANZAR DASHBOARD
public_url = ngrok.connect(8501)
print(f"\nDASHBOARD EN VIVO:\n{public_url}\n")

!streamlit run /content/mechera_dashboard.py --server.port 8501


DASHBOARD EN VIVO:
NgrokTunnel: "https://triste-pensive-jeannette.ngrok-free.dev" -> "http://localhost:8501"




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.169.73.150:8501

